In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import os
import shap
from lime import lime_tabular
import pickle

# 생존 분석 라이브러리
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import joblib

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

def setup_korean_font():
    """한글 폰트 설정"""
    import platform
    import matplotlib.font_manager as fm
    
    system = platform.system()
    
    if system == 'Windows':
        try:
            plt.rcParams['font.family'] = 'Malgun Gothic'
        except:
            try:
                font_path = 'C:/Windows/Fonts/malgun.ttf'
                font_name = fm.FontProperties(fname=font_path).get_name()
                plt.rc('font', family=font_name)
            except:
                print("⚠️ 한글 폰트 설정 실패")
    elif system == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    
    plt.rcParams['axes.unicode_minus'] = False
    print("✅ 한글 폰트 설정 완료")

class KidneyCancerTreatmentEffectPredictorCDSS:
    """신장암 치료 효과 예측 모델 클래스 (CDSS 호환 + XAI + 과적합 방지)"""
    
    def __init__(self, data_path):
        setup_korean_font()
        self.data_path = data_path
        self.df = None
        self.processed_df = None
        self.models = {}
        self.results = {}
        self.feature_names = []
        self.scaler = None
        self.label_encoders = {}
        self.shap_explainers = {}
        self.shap_values = {}
        self.lime_explainers = {}
        self.holdout_patient = None
        
        print(f"🚀 신장암 치료 효과 예측 모델 초기화 (CDSS 호환 + XAI + 과적합 방지)")
        print(f"📁 데이터 경로: {data_path}")
        print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*60)
    
    def load_and_explore_data(self):
        """데이터 로드 및 탐색적 분석"""
        print("\n📊 1. 데이터 로드 및 탐색")
        
        try:
            self.df = pd.read_csv(self.data_path)
            print(f"✅ 데이터 로드 성공: {self.df.shape[0]}행 × {self.df.shape[1]}열")
        except Exception as e:
            print(f"❌ 데이터 로드 실패: {e}")
            return False
        
        # 기본 정보 출력
        print(f"📈 데이터 기본 정보:")
        print(f"   - 총 환자 수: {len(self.df)}")
        print(f"   - 총 컬럼 수: {len(self.df.columns)}")
        
        # 생존 상태 분포
        if 'vital_status' in self.df.columns:
            status_counts = self.df['vital_status'].value_counts()
            print(f"   - 생존 환자: {status_counts.get('Alive', 0)}명")
            print(f"   - 사망 환자: {status_counts.get('Dead', 0)}명")
            print(f"   - 사망률: {status_counts.get('Dead', 0)/len(self.df)*100:.1f}%")
        
        return True
    
    def preprocess_data(self):
        """데이터 전처리"""
        print("\n🔧 2. 데이터 전처리")
        
        # 신장암 치료 효과 예측 모델용 선택된 컬럼들
        selected_columns = [
            # 생존 결과 변수
            'vital_status', 'days_to_death', 'days_to_last_follow_up',
            
            # 치료 관련 변수 (가장 중요)
            'treatments_pharmaceutical_treatment_type',
            'treatments_pharmaceutical_treatment_intent_type',
            'treatments_pharmaceutical_treatment_or_therapy',
            'treatments_radiation_treatment_type',
            'treatments_radiation_treatment_or_therapy',
            
            # 병기 및 종양 특성 (예후 인자)
            'ajcc_pathologic_stage', 'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_m',
            'morphology', 'primary_diagnosis',
            
            # 환자 기본 정보
            'age_at_diagnosis', 'gender', 'race', 'ethnicity',
            
            # 이전 치료력
            'prior_treatment', 'prior_malignancy',
            
            # 생활습관 위험인자
            'tobacco_smoking_status', 'pack_years_smoked',
            
            # 해부학적/종양 특성
            'laterality', 'site_of_resection_or_biopsy', 'tissue_or_organ_of_origin',
            'synchronous_malignancy',
            
            # 시간 관련 변수
            'days_to_diagnosis', 'year_of_diagnosis'
        ]
        
        # 존재하는 컬럼만 선택
        available_columns = [col for col in selected_columns if col in self.df.columns]
        missing_columns = [col for col in selected_columns if col not in self.df.columns]
        
        print(f"✅ 사용 가능한 컬럼: {len(available_columns)}개")
        if missing_columns:
            print(f"⚠️  누락된 컬럼: {missing_columns}")
        
        self.processed_df = self.df[available_columns].copy()
        
        # Series 객체를 단일 값으로 변환하는 함수
        def extract_first_value(x):
            if isinstance(x, pd.Series):
                return x.iloc[0] if len(x) > 0 else np.nan
            else:
                return x
        
        # days_to_last_follow_up 컬럼 내 Series 객체를 단일 값으로 변환
        if 'days_to_last_follow_up' in self.processed_df.columns:
            self.processed_df['days_to_last_follow_up'] = self.processed_df['days_to_last_follow_up'].apply(extract_first_value)
        
        # 생존 시간 및 이벤트 변수 생성
        print("🔄 생존 변수 생성 중...")
        self.processed_df['event'] = (self.processed_df['vital_status'] == 'Dead').astype(int)
        
        # 생존 시간 계산 (Series 처리 포함)
        def calculate_duration(row):
            death_day = row['days_to_death']
            followup_day = row['days_to_last_follow_up']
            
            # followup_day가 Series인지 확인하고 처리
            if isinstance(followup_day, pd.Series):
                followup_day = followup_day.iloc[0] if len(followup_day) > 0 else np.nan
            
            if pd.notna(death_day):
                return death_day
            elif pd.notna(followup_day):
                return followup_day
            else:
                return np.nan
        
        self.processed_df['duration'] = self.processed_df.apply(calculate_duration, axis=1)
        
        # 치료 효과 지표 생성 (신장암 특화)
        self.processed_df['treatment_effectiveness'] = np.where(
            self.processed_df['duration'] > self.processed_df['duration'].median(), 1, 0
        )
        
        # 유효하지 않은 생존 시간 제거
        valid_mask = (pd.notna(self.processed_df['duration'])) & (self.processed_df['duration'] > 0)
        self.processed_df = self.processed_df[valid_mask].copy()
        
        print(f"✅ 유효한 생존 데이터: {len(self.processed_df)}명")
        print(f"   - 사망 이벤트: {self.processed_df['event'].sum()}건")
        print(f"   - 중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        print(f"   - 치료 효과 양호: {self.processed_df['treatment_effectiveness'].sum()}명")
        
        # 결측값 분석
        print("\n📋 결측값 분석:")
        missing_analysis = self.processed_df.isnull().sum()
        missing_percent = (missing_analysis / len(self.processed_df) * 100).round(1)
        
        for col in missing_analysis[missing_analysis > 0].index:
            print(f"   - {col}: {missing_analysis[col]}개 ({missing_percent[col]}%)")
        
        # 높은 결측률 컬럼 제거 (80% 이상)
        high_missing_cols = missing_percent[missing_percent > 80].index.tolist()
        if high_missing_cols:
            print(f"🗑️  높은 결측률 컬럼 제거: {high_missing_cols}")
            self.processed_df = self.processed_df.drop(columns=high_missing_cols)
        
        return True
    
    def prepare_features(self):
        """특성 준비 및 인코딩 (CDSS 호환)"""
        print("\n🎯 3. 특성 준비 및 인코딩")
        
        # CDSS 테스트용 환자 1명 미리 분리
        print("🔄 CDSS 테스트용 환자 분리 중...")
        holdout_idx = self.processed_df.sample(n=1, random_state=42).index[0]
        self.holdout_patient = self.processed_df.loc[holdout_idx:holdout_idx].copy()
        remaining_df = self.processed_df.drop(holdout_idx).copy()
        
        print(f"   - CDSS 테스트 환자: {holdout_idx}")
        print(f"   - 모델 훈련용 데이터: {len(remaining_df)}명")
        
        # 특성과 타겟 분리
        feature_cols = [col for col in remaining_df.columns 
                       if col not in ['vital_status', 'days_to_death', 'days_to_last_follow_up', 
                                     'event', 'duration', 'treatment_effectiveness']]
        
        X = remaining_df[feature_cols].copy()
        y_duration = remaining_df['duration'].values
        y_event = remaining_df['event'].values.astype(bool)
        y_effectiveness = remaining_df['treatment_effectiveness'].values
        
        print(f"📊 초기 특성 개수: {len(feature_cols)}")
        print(f"📊 샘플 개수: {len(X)}")
        
        # 범주형 변수 인코딩
        categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()
        
        print(f"🔤 범주형 변수: {len(categorical_cols)}개")
        print(f"🔢 수치형 변수: {len(numerical_cols)}개")
        
        # 결측값 처리
        print("🔄 결측값 처리 중...")
        
        # 임상적으로 의미있는 Unknown 값을 가질 수 있는 컬럼들 (신장암 특화)
        meaningful_unknown_cols = [
            'ajcc_pathologic_stage', 'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_m',
            'morphology', 'primary_diagnosis',
            'treatments_pharmaceutical_treatment_type', 'treatments_pharmaceutical_treatment_intent_type',
            'treatments_pharmaceutical_treatment_or_therapy', 'treatments_radiation_treatment_type',
            'treatments_radiation_treatment_or_therapy', 'prior_treatment', 'prior_malignancy',
            'synchronous_malignancy'
        ]
        
        self.label_encoders = {}
        
        for col in categorical_cols:
            if col in X.columns:
                print(f"\n   🔍 {col} 처리:")
                
                # 현재 값 분포 확인
                value_counts = X[col].value_counts(dropna=False)
                print(f"      - 전처리 전 분포: {dict(list(value_counts.items())[:3])}")
                
                # 'NA' 문자열을 결측치로 변환
                if 'NA' in X[col].values:
                    X[col] = X[col].replace('NA', np.nan)
                    print(f"      - 'NA' 문자열을 결측치로 변환")
                
                # Unknown 값 처리 결정
                has_unknown = X[col].str.contains('Unknown', na=False).any() if X[col].dtype == object else False
                
                if has_unknown:
                    if col in meaningful_unknown_cols:
                        print(f"      - 'Unknown' 값 유지 (임상적 의미 있음)")
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                    else:
                        print(f"      - 'Unknown' 값을 결측치로 변환 후 대체")
                        X[col] = X[col].replace('Unknown', np.nan)
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                else:
                    if X[col].isnull().any():
                        mode_value = X[col].mode()
                        if not mode_value.empty:
                            fill_value = mode_value[0]
                            X[col] = X[col].fillna(fill_value)
                            print(f"      - 결측치를 '{fill_value}'로 대체")
        
        # 모든 범주형 변수 인코딩
        print("\n🔄 범주형 변수 인코딩:")
        all_categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        for col in all_categorical_cols:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].astype(str))
            self.label_encoders[col] = le
            
            if col in meaningful_unknown_cols:
                mapping = dict(zip(le.classes_, le.transform(le.classes_)))
                print(f"   - {col} 인코딩 매핑: {mapping}")
        
        # 모든 특성을 수치형으로 변환
        for col in X.columns:
            X[col] = pd.to_numeric(X[col], errors='coerce')
        
        # 전체 특성에 대해 Imputer 훈련
        print(f"\n📏 전체 특성 Imputer 훈련:")
        print(f"   - 전체 특성 수: {X.shape[1]}")
        
        self.num_imputer = SimpleImputer(strategy='median')
        X_imputed = self.num_imputer.fit_transform(X)
        X = pd.DataFrame(X_imputed, columns=X.columns, index=X.index)
        
        print(f"✅ 전체 특성 Imputer 훈련 완료: {X.shape[1]}개 특성")
        
        # 특성 스케일링
        self.scaler = StandardScaler()
        X_scaled = pd.DataFrame(
            self.scaler.fit_transform(X),
            columns=X.columns,
            index=X.index
        )
        
        self.feature_names = X_scaled.columns.tolist()
        
        # scikit-survival 형식으로 변환
        y_structured = np.array([(event, duration) for event, duration in zip(y_event, y_duration)],
                               dtype=[('event', '?'), ('time', '<f8')])
        
        print("✅ 특성 준비 완료 (CDSS 호환)")
        
        return X_scaled, y_structured, y_duration, y_event, y_effectiveness
    
    def split_data(self, X, y_structured, y_duration, y_event, y_effectiveness):
        """데이터 분할"""
        print("\n✂️  4. 데이터 분할 (훈련:검증:테스트 = 60:20:20)")
        
        # 먼저 훈련+검증 vs 테스트로 분할
        X_temp, X_test, y_temp_struct, y_test_struct, y_temp_dur, y_test_dur, y_temp_event, y_test_event, y_temp_eff, y_test_eff = \
            train_test_split(X, y_structured, y_duration, y_event, y_effectiveness,
                           test_size=0.2, random_state=42, stratify=y_effectiveness)
        
        # 훈련 vs 검증으로 분할
        X_train, X_val, y_train_struct, y_val_struct, y_train_dur, y_val_dur, y_train_event, y_val_event, y_train_eff, y_val_eff = \
            train_test_split(X_temp, y_temp_struct, y_temp_dur, y_temp_event, y_temp_eff,
                           test_size=0.25, random_state=42, stratify=y_temp_eff)
        
        print(f"📊 훈련 세트: {len(X_train)}명 (사망: {y_train_event.sum()}명, 치료효과양호: {y_train_eff.sum()}명)")
        print(f"📊 검증 세트: {len(X_val)}명 (사망: {y_val_event.sum()}명, 치료효과양호: {y_val_eff.sum()}명)")
        print(f"📊 테스트 세트: {len(X_test)}명 (사망: {y_test_event.sum()}명, 치료효과양호: {y_test_eff.sum()}명)")
        print(f"📊 CDSS 테스트: 1명 (별도 보관)")
        
        return (X_train, X_val, X_test, 
                y_train_struct, y_val_struct, y_test_struct,
                y_train_dur, y_val_dur, y_test_dur,
                y_train_event, y_val_event, y_test_event,
                y_train_eff, y_val_eff, y_test_eff)
    
    def train_models(self, X_train, X_val, X_test, 
                    y_train_struct, y_val_struct, y_test_struct,
                    y_train_dur, y_val_dur, y_test_dur,
                    y_train_event, y_val_event, y_test_event,
                    y_train_eff, y_val_eff, y_test_eff):
        """모델 훈련 (균형잡힌 과적합 방지)"""
        print("\n🤖 5. 모델 훈련 (균형잡힌 과적합 방지)")
        
        # 1. Random Survival Forest (생존 예측)
        print("🔄 Random Survival Forest 훈련 중...")
        try:
            rsf_model = RandomSurvivalForest(
                n_estimators=100,
                max_depth=8,
                min_samples_split=15,
                min_samples_leaf=8,
                random_state=42,
                n_jobs=-1
            )
            rsf_model.fit(X_train, y_train_struct)
            
            rsf_wrapper = {
                'model': rsf_model,
                'model_type': 'RandomSurvivalForest',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'event_rate': y_train_event.mean()
                }
            }
            
            self.models['RSF'] = rsf_wrapper
            print("✅ Random Survival Forest 훈련 완료")
        except Exception as e:
            print(f"❌ RSF 모델 훈련 실패: {e}")
        
        # 2. Cox 비례위험 모델 (생존 예측)
        print("🔄 Cox 비례위험 모델 훈련 중...")
        try:
            cox_model = CoxPHSurvivalAnalysis(alpha=0.5)
            cox_model.fit(X_train, y_train_struct)
            
            cox_wrapper = {
                'model': cox_model,
                'model_type': 'CoxPHSurvivalAnalysis',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'event_rate': y_train_event.mean()
                }
            }
            
            self.models['Cox_Survival'] = cox_wrapper
            print("✅ Cox 생존 모델 훈련 완료")
        except Exception as e:
            print(f"❌ Cox 모델 훈련 실패: {e}")
        
        # 3. Random Forest (균형잡힌 설정)
        print("🔄 Random Forest 균형잡힌 설정으로 훈련 중...")
        try:
            rf_classifier = RandomForestClassifier(
                n_estimators=100,
                max_depth=8,
                min_samples_split=15,
                min_samples_leaf=8,
                max_features='sqrt',
                random_state=42,
                n_jobs=-1
            )
            rf_classifier.fit(X_train, y_train_eff)
            
            rf_wrapper = {
                'model': rf_classifier,
                'model_type': 'RandomForestClassifier',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'class_labels': ['치료효과 불량', '치료효과 양호'],
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'class_distribution': np.bincount(y_train_eff),
                    'feature_importance': rf_classifier.feature_importances_
                }
            }
            
            self.models['RF_Treatment'] = rf_wrapper
            print("✅ Random Forest 균형잡힌 훈련 완료")
        except Exception as e:
            print(f"❌ RF 모델 훈련 실패: {e}")
        
        # 4. XGBoost (적절한 정규화)
        print("🔄 XGBoost 적절한 정규화로 훈련 중...")
        try:
            xgb_model = xgb.XGBClassifier(
                n_estimators=100,
                max_depth=5,
                learning_rate=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.5,
                reg_lambda=0.5,
                random_state=42,
                eval_metric='logloss'
            )
            
            xgb_model.fit(X_train, y_train_eff)
            
            xgb_wrapper = {
                'model': xgb_model,
                'model_type': 'XGBClassifier',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'class_labels': ['치료효과 불량', '치료효과 양호'],
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'class_distribution': np.bincount(y_train_eff),
                    'feature_importance': xgb_model.feature_importances_
                }
            }
            
            self.models['XGB_Treatment'] = xgb_wrapper
            print("✅ XGBoost 적절한 정규화 훈련 완료")
        except Exception as e:
            print(f"❌ XGBoost 모델 훈련 실패: {e}")
        
        # 5. LightGBM (과적합 방지 최적화)
        print("🔄 LightGBM 과적합 방지 최적화로 훈련 중...")
        try:
            lgb_model = lgb.LGBMClassifier(
                n_estimators=100,
                max_depth=4,
                learning_rate=0.05,
                num_leaves=15,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.5,
                reg_lambda=0.5,
                min_child_samples=20,
                min_child_weight=0.001,
                random_state=42,
                verbose=-1
            )
            
            lgb_model.fit(
                X_train, y_train_eff,
                eval_set=[(X_val, y_val_eff)],
                callbacks=[lgb.early_stopping(10), lgb.log_evaluation(0)]
            )
            
            lgb_wrapper = {
                'model': lgb_model,
                'model_type': 'LGBMClassifier',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'class_labels': ['치료효과 불량', '치료효과 양호'],
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'class_distribution': np.bincount(y_train_eff),
                    'feature_importance': lgb_model.feature_importances_
                }
            }
            
            self.models['LGB_Treatment'] = lgb_wrapper
            print("✅ LightGBM 과적합 방지 훈련 완료")
        except Exception as e:
            print(f"❌ LightGBM 모델 훈련 실패: {e}")
        
        print(f"\n🎯 균형잡힌 {len(self.models)}개 모델 훈련 완료")
        
        return True
    
    def evaluate_models(self, X_train, X_val, X_test,
                       y_train_struct, y_val_struct, y_test_struct,
                       y_train_dur, y_val_dur, y_test_dur,
                       y_train_event, y_val_event, y_test_event,
                       y_train_eff, y_val_eff, y_test_eff):
        """모델 평가"""
        print("\n📈 6. 모델 평가")
        
        datasets = {
            'Train': (X_train, y_train_struct, y_train_dur, y_train_event, y_train_eff),
            'Validation': (X_val, y_val_struct, y_val_dur, y_val_event, y_val_eff),
            'Test': (X_test, y_test_struct, y_test_dur, y_test_event, y_test_eff)
        }
        
        for model_name, model_wrapper in self.models.items():
            print(f"\n🔍 {model_name} 모델 평가:")
            self.results[model_name] = {}
            
            for dataset_name, (X, y_struct, y_dur, y_event, y_eff) in datasets.items():
                try:
                    actual_model = model_wrapper['model']
                    model_type = model_wrapper['model_type']
                    
                    if model_type in ['RandomSurvivalForest', 'CoxPHSurvivalAnalysis']:
                        # 생존 모델 평가
                        risk_scores = actual_model.predict(X)
                        c_index = concordance_index_censored(y_struct['event'], y_struct['time'], risk_scores)[0]
                        self.results[model_name][dataset_name] = {'c_index': c_index}
                        print(f"   {dataset_name}: C-index = {c_index:.3f}")
                    
                    else:
                        # 분류 모델 평가 (치료 효과)
                        y_pred = actual_model.predict(X)
                        accuracy = (y_pred == y_eff).mean()
                        
                        try:
                            y_proba = actual_model.predict_proba(X)
                            auc_score = roc_auc_score(y_eff, y_proba[:, 1])
                        except:
                            auc_score = np.nan
                        
                        self.results[model_name][dataset_name] = {
                            'accuracy': accuracy,
                            'auc': auc_score
                        }
                        print(f"   {dataset_name}: Accuracy = {accuracy:.3f}, AUC = {auc_score:.3f}")
                    
                except Exception as e:
                    print(f"   ❌ {dataset_name} 평가 실패: {e}")
                    self.results[model_name][dataset_name] = {'error': str(e)}
        
        return True
    
    def explain_models(self, X_train, X_test):
        """XAI 모델 설명 생성"""
        print("\n🔍 XAI 모델 설명 생성")
        
        # SHAP 설명기 초기화
        print("🔄 SHAP 설명 생성 중...")
        tree_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']
        
        for model_name in tree_models:
            if model_name in self.models:
                try:
                    X_test_sample = X_test.iloc[:50]
                    
                    actual_model = self.models[model_name]['model']
                    
                    explainer = shap.TreeExplainer(actual_model)
                    shap_values = explainer.shap_values(X_test_sample)
                    
                    self.shap_explainers[model_name] = explainer
                    self.shap_values[model_name] = shap_values
                    print(f"✅ {model_name} SHAP 설명 생성 완료")
                except Exception as e:
                    print(f"❌ {model_name} SHAP 실패: {e}")
        
        # LIME 설명기 초기화
        print("\n🔄 LIME 설명 생성 중...")
        classification_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']
        
        for model_name in classification_models:
            if model_name in self.models:
                try:
                    actual_model = self.models[model_name]['model']
                    class_labels = self.models[model_name]['class_labels']
                    
                    explainer = lime_tabular.LimeTabularExplainer(
                        training_data=X_train.values,
                        feature_names=self.feature_names,
                        class_names=class_labels,
                        mode='classification',
                        discretize_continuous=True
                    )
                    
                    self.lime_explainers[model_name] = {
                        'explainer': explainer,
                        'predict_fn': actual_model.predict_proba
                    }
                    print(f"✅ {model_name} LIME 설명기 생성 완료")
                except Exception as e:
                    print(f"❌ {model_name} LIME 실패: {e}")
        
        return True
    
    def generate_xai_visualizations(self, X_test, sample_index=0):
        """XAI 시각화 생성 및 저장"""
        print("\n📊 XAI 시각화 생성")
        
        # SHAP 시각화
        shap_figures = []
        for model_name in self.shap_explainers:
            try:
                print(f"🔄 {model_name} SHAP 시각화 생성 중...")
                
                shap_vals = self.shap_values[model_name]
                
                # 이진 분류 SHAP 값 처리
                if isinstance(shap_vals, list) and len(shap_vals) == 2:
                    shap_vals_to_plot = shap_vals[1]
                    expected_val = self.shap_explainers[model_name].expected_value[1]
                elif hasattr(shap_vals, 'shape') and shap_vals.ndim == 3:
                    shap_vals_to_plot = shap_vals[:, :, 1]
                    expected_val = (self.shap_explainers[model_name].expected_value[1] 
                                if hasattr(self.shap_explainers[model_name].expected_value, '__len__') 
                                else self.shap_explainers[model_name].expected_value)
                else:
                    shap_vals_to_plot = shap_vals
                    expected_val = (self.shap_explainers[model_name].expected_value 
                                if not hasattr(self.shap_explainers[model_name].expected_value, '__len__')
                                else self.shap_explainers[model_name].expected_value[0])
                
                # Summary plot
                plt.figure(figsize=(10,6))
                shap.summary_plot(shap_vals_to_plot, X_test.iloc[:50], 
                                feature_names=self.feature_names,
                                plot_type="bar", show=False)
                plt.title(f"{model_name} 특성 중요도 (SHAP)")
                shap_summary_path = f"shap_summary_{model_name}_kidney.png"
                plt.savefig(shap_summary_path, bbox_inches='tight')
                plt.close()
                
                # Individual waterfall plot
                plt.figure(figsize=(12,6))
                
                if shap_vals_to_plot.ndim == 2:
                    individual_shap = shap_vals_to_plot[sample_index, :]
                else:
                    individual_shap = shap_vals_to_plot
                
                if hasattr(expected_val, '__len__') and len(expected_val) > 0:
                    base_value = float(expected_val[0]) if hasattr(expected_val[0], '__float__') else 0.0
                else:
                    base_value = float(expected_val) if hasattr(expected_val, '__float__') else 0.0
                
                try:
                    shap.waterfall_plot(
                        shap.Explanation(
                            values=individual_shap.astype(float),
                            base_values=base_value,
                            data=X_test.iloc[sample_index].values.astype(float),
                            feature_names=self.feature_names
                        ),
                        show=False
                    )
                    plt.title(f"{model_name} 개별 설명 (샘플 {sample_index})")
                    shap_waterfall_path = f"shap_waterfall_{model_name}_kidney_{sample_index}.png"
                    plt.savefig(shap_waterfall_path, bbox_inches='tight')
                    plt.close()
                    
                    shap_figures.extend([shap_summary_path, shap_waterfall_path])
                    
                except Exception as waterfall_error:
                    print(f"⚠️ Waterfall plot 실패, Force plot으로 대체: {waterfall_error}")
                    
                    shap.force_plot(
                        base_value,
                        individual_shap,
                        X_test.iloc[sample_index],
                        feature_names=self.feature_names,
                        matplotlib=True,
                        show=False
                    )
                    plt.title(f"{model_name} 개별 설명 - Force Plot (샘플 {sample_index})")
                    shap_force_path = f"shap_force_{model_name}_kidney_{sample_index}.png"
                    plt.savefig(shap_force_path, bbox_inches='tight')
                    plt.close()
                    
                    shap_figures.extend([shap_summary_path, shap_force_path])
                
                print(f"✅ {model_name} SHAP 시각화 완료")
                
            except Exception as e:
                print(f"❌ {model_name} SHAP 시각화 실패: {e}")
        
        # LIME 시각화
        lime_figures = []
        for model_name in self.lime_explainers:
            try:
                lime_data = self.lime_explainers[model_name]
                exp = lime_data['explainer'].explain_instance(
                    X_test.iloc[sample_index].values,
                    lime_data['predict_fn'],
                    num_features=5
                )
                lime_path = f"lime_explanation_{model_name}_kidney_{sample_index}.png"
                fig = exp.as_pyplot_figure()
                plt.title(f"{model_name} LIME 설명 (샘플 {sample_index})")
                plt.savefig(lime_path, bbox_inches='tight')
                plt.close()
                lime_figures.append(lime_path)
                print(f"✅ {model_name} LIME 시각화 완료")
            except Exception as e:
                print(f"❌ {model_name} LIME 시각화 실패: {e}")
        
        return shap_figures, lime_figures
    
    def preprocess_holdout_patient(self):
        """Holdout 환자 데이터 전처리 (더 안전한 방법)"""
        print("\n🔧 Holdout 환자 전처리 시작")
        
        # holdout 환자의 특성 데이터만 추출
        feature_cols = [col for col in self.holdout_patient.columns 
                       if col not in ['vital_status', 'days_to_death', 'days_to_last_follow_up', 
                                     'event', 'duration', 'treatment_effectiveness']]
        
        patient_raw = self.holdout_patient[feature_cols].copy()
        print(f"🔍 원본 환자 특성: {len(patient_raw.columns)}개")
        print(f"🔍 모델 훈련 특성: {len(self.feature_names)}개")
        
        # 모델 훈련 시 사용한 특성명과 정확히 일치하는 DataFrame 생성
        patient_processed = pd.DataFrame(index=patient_raw.index)
        
        for feature_name in self.feature_names:
            if feature_name in patient_raw.columns:
                patient_processed[feature_name] = patient_raw[feature_name].copy()
                print(f"✅ {feature_name}: 원본 데이터 사용")
            else:
                patient_processed[feature_name] = 0.0
                print(f"⚠️ {feature_name}: 기본값(0.0) 설정")
        
        # 범주형 변수 전처리
        print("\n🔄 범주형 변수 전처리:")
        for col, encoder in self.label_encoders.items():
            if col in patient_processed.columns:
                try:
                    original_value = patient_processed[col].iloc[0]
                    print(f"   - {col}: 원본값 = {original_value}")
                    
                    if pd.isna(original_value) or original_value == 'NA':
                        encoded_value = encoder.transform([encoder.classes_[0]])[0]
                        patient_processed[col] = float(encoded_value)
                        print(f"     → 결측치를 '{encoder.classes_[0]}' ({encoded_value})로 대체")
                    else:
                        str_value = str(original_value)
                        if str_value in encoder.classes_:
                            encoded_value = encoder.transform([str_value])[0]
                            patient_processed[col] = float(encoded_value)
                            print(f"     → 인코딩: '{str_value}' → {encoded_value}")
                        else:
                            encoded_value = encoder.transform([encoder.classes_[0]])[0]
                            patient_processed[col] = float(encoded_value)
                            print(f"     → 새로운 값 '{str_value}'을 '{encoder.classes_[0]}' ({encoded_value})로 대체")
                            
                except Exception as e:
                    print(f"     ❌ {col} 인코딩 실패: {e}")
                    patient_processed[col] = 0.0
        
        # 모든 컬럼을 수치형으로 변환
        print("\n🔢 데이터 타입 변환:")
        for col in patient_processed.columns:
            try:
                if pd.api.types.is_numeric_dtype(patient_processed[col]):
                    print(f"   - {col}: 이미 숫자형")
                else:
                    patient_processed[col] = pd.to_numeric(patient_processed[col], errors='coerce')
                    print(f"   - {col}: 숫자형으로 변환")
                
                if patient_processed[col].isnull().any():
                    patient_processed[col] = patient_processed[col].fillna(0.0)
                    print(f"   - {col}: 결측값을 0.0으로 대체")
                    
            except Exception as e:
                print(f"⚠️ {col} 수치형 변환 실패: {e}")
                patient_processed[col] = 0.0
        
        # 특성 순서를 훈련 시와 정확히 일치시키기
        patient_processed = patient_processed[self.feature_names]
        print(f"✅ 특성 순서 정렬 완료: {patient_processed.shape}")
        print(f"✅ 특성명 확인: {list(patient_processed.columns) == self.feature_names}")
        
        # Imputer 적용 (안전한 방법)
        if hasattr(self, 'num_imputer') and self.num_imputer is not None:
            try:
                print(f"\n📏 Imputer 적용 (안전한 방법):")
                print(f"   - 입력 형태: {patient_processed.shape}")
                
                if hasattr(self.num_imputer, 'n_features_in_'):
                    expected_features = self.num_imputer.n_features_in_
                    print(f"   - Imputer 기대 특성 수: {expected_features}")
                    
                    if patient_processed.shape[1] == expected_features:
                        patient_values = patient_processed.values
                        imputed_values = self.num_imputer.transform(patient_values)
                        patient_processed = pd.DataFrame(
                            imputed_values,
                            columns=self.feature_names,
                            index=patient_processed.index
                        )
                        print(f"✅ Imputer 적용 완료")
                    else:
                        print(f"⚠️ 특성 수 불일치: 입력 {patient_processed.shape[1]} vs 기대 {expected_features}")
                        print(f"⚠️ Imputer 건너뜀 - 결측값은 이미 처리됨")
                else:
                    print(f"⚠️ Imputer 특성 정보 없음 - 안전하게 건너뜀")
                    
            except Exception as e:
                print(f"⚠️ Imputer 적용 중 오류 발생, 건너뜀: {e}")
        else:
            print(f"ℹ️ Imputer가 없거나 None - 건너뜀")
        
        # Scaler 적용 (안전한 방법)
        if hasattr(self, 'scaler') and self.scaler is not None:
            try:
                print(f"\n📏 스케일링 적용 (안전한 방법):")
                print(f"   - 입력 형태: {patient_processed.shape}")
                
                if hasattr(self.scaler, 'n_features_in_'):
                    expected_features = self.scaler.n_features_in_
                    print(f"   - Scaler 기대 특성 수: {expected_features}")
                    
                    if patient_processed.shape[1] == expected_features:
                        patient_values = patient_processed.values
                        scaled_values = self.scaler.transform(patient_values)
                        patient_features_scaled = pd.DataFrame(
                            scaled_values,
                            columns=self.feature_names,
                            index=patient_processed.index
                        )
                        print(f"✅ 스케일링 완료")
                        print(f"✅ 최종 특성 형태: {patient_features_scaled.shape}")
                        return patient_features_scaled
                    else:
                        print(f"⚠️ 특성 수 불일치: 입력 {patient_processed.shape[1]} vs 기대 {expected_features}")
                        print(f"⚠️ 스케일링 없이 반환")
                        return patient_processed
                else:
                    print(f"⚠️ Scaler 특성 정보 없음 - 안전하게 건너뜀")
                    return patient_processed
                    
            except Exception as e:
                print(f"❌ 스케일링 중 오류 발생: {e}")
                print(f"⚠️ 스케일링 없이 반환")
                return patient_processed
        else:
            print(f"ℹ️ Scaler가 없거나 None - 건너뜀")
            return patient_processed
    
    def test_cdss_compatibility(self):
        """CDSS 호환성 테스트"""
        print("\n🔬 CDSS 호환성 테스트")
        
        try:
            # holdout 환자 데이터 전처리
            holdout_features = self.preprocess_holdout_patient()
            
            # 각 모델로 예측 수행
            predictions = {}
            
            for model_name, model_wrapper in self.models.items():
                try:
                    print(f"\n🔄 {model_name} 모델 예측 중...")
                    
                    actual_model = model_wrapper['model']
                    model_type = model_wrapper['model_type']
                    
                    print(f"   - 모델 타입: {model_type}")
                    print(f"   - 입력 특성 수: {holdout_features.shape[1]}")
                    
                    if model_type in ['RandomSurvivalForest', 'CoxPHSurvivalAnalysis']:
                        # 생존 모델
                        pred = actual_model.predict(holdout_features)[0]
                        predictions[model_name] = {
                            'type': 'survival',
                            'risk_score': pred
                        }
                        print(f"✅ {model_name}: 위험도 점수 = {pred:.4f}")
                    else:
                        # 분류 모델
                        pred_class = actual_model.predict(holdout_features)[0]
                        pred_proba = actual_model.predict_proba(holdout_features)[0]
                        class_labels = model_wrapper['class_labels']
                        
                        predictions[model_name] = {
                            'type': 'classification',
                            'class': pred_class,
                            'class_name': class_labels[pred_class],
                            'probabilities': pred_proba
                        }
                        
                        print(f"✅ {model_name}: {class_labels[pred_class]} (확률: {pred_proba[pred_class]:.3f})")
                    
                except Exception as e:
                    print(f"❌ {model_name} 예측 실패: {e}")
                    predictions[model_name] = {'error': str(e)}
            
            return predictions
            
        except Exception as e:
            print(f"❌ CDSS 호환성 테스트 실패: {e}")
            return {}
    
    def save_models_for_cdss(self):
        """CDSS 호환 모델 저장"""
        print("\n💾 CDSS 호환 모델 저장")
        
        # 전체 파이프라인을 하나의 객체로 저장
        cdss_pipeline = {
            'models': self.models,
            'holdout_patient': self.holdout_patient,
            'metadata': {
                'created_date': datetime.now().isoformat(),
                'model_version': '4.0',
                'description': 'TCGA-KIRC 신장암 치료 효과 예측 모델 (CDSS 호환 + 과적합 방지)',
                'class_labels': ['치료효과 불량', '치료효과 양호']
            }
        }
        
        # 개별 모델도 저장
        for model_name, model_wrapper in self.models.items():
            try:
                filename = f"cdss_kidney_cancer_treatment_{model_name.lower()}_model.pkl"
                with open(filename, 'wb') as f:
                    pickle.dump(model_wrapper, f)
                print(f"✅ {model_name} 모델 저장: {filename}")
            except Exception as e:
                print(f"❌ {model_name} 모델 저장 실패: {e}")
        
        # 전체 파이프라인 저장
        try:
            pipeline_filename = "cdss_kidney_cancer_treatment_complete_pipeline.pkl"
            with open(pipeline_filename, 'wb') as f:
                pickle.dump(cdss_pipeline, f)
            print(f"✅ 전체 파이프라인 저장: {pipeline_filename}")
        except Exception as e:
            print(f"❌ 파이프라인 저장 실패: {e}")
        
        return True
    
    def plot_results(self, X_test, y_test_dur, y_test_event, y_test_eff):
        """결과 시각화 (치료 효과 예측 특화)"""
        print("\n📊 7. 결과 시각화")
        
        fig = plt.figure(figsize=(25, 35))
        gs = fig.add_gridspec(5, 3)
        axes = [
            fig.add_subplot(gs[0, 0]),  # 치료효과별 생존 곡선
            fig.add_subplot(gs[0, 1]),  # 치료효과 분포
            fig.add_subplot(gs[0, 2]),  # 생존 모델 성능
            fig.add_subplot(gs[1, 0]),  # 치료효과 분류 성능
            fig.add_subplot(gs[1, 1]),  # AUC 성능
            fig.add_subplot(gs[1, 2]),  # 특성 중요도 (XGBoost)
            fig.add_subplot(gs[2, 0]),  # 특성 중요도 (LightGBM)
            fig.add_subplot(gs[2, 1]),  # 치료효과별 생존율
            fig.add_subplot(gs[2, 2]),  # CDSS 테스트 결과
            fig.add_subplot(gs[3, :]),  # SHAP 시각화
            fig.add_subplot(gs[4, :])   # LIME 시각화
        ]
        
        fig.suptitle('신장암 치료 효과 예측 모델 분석 결과 (CDSS 호환 + XAI + 과적합 방지)', fontsize=16, fontweight='bold')
        
        # 1. 치료효과별 생존 곡선
        print("🔍 치료효과별 생존 곡선 생성 중...")
        for eff_level in [0, 1]:
            mask = (y_test_eff == eff_level)
            if mask.sum() > 5:
                kmf = KaplanMeierFitter()
                label = '치료효과 양호' if eff_level == 1 else '치료효과 불량'
                kmf.fit(y_test_dur[mask], y_test_event[mask], label=f'{label} (n={mask.sum()})')
                kmf.plot_survival_function(ax=axes[0])
        
        axes[0].set_title('치료효과별 생존 곡선 (Kaplan-Meier)')
        axes[0].set_ylabel('생존 확률')
        axes[0].set_xlabel('시간 (일)')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # 2. 치료효과 분포
        eff_counts = pd.Series(y_test_eff).value_counts().sort_index()
        colors = ['lightcoral', 'lightgreen']
        labels = ['치료효과 불량', '치료효과 양호']
        bars = axes[1].bar([labels[i] for i in eff_counts.index], 
                          eff_counts.values, color=colors)
        axes[1].set_title('테스트 세트 치료효과 분포')
        axes[1].set_ylabel('환자 수')
        
        for bar, value in zip(bars, eff_counts.values):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                        f'{value}명', ha='center', va='bottom', fontweight='bold')
        
        # 3. 생존 모델 성능 비교
        survival_models = ['RSF', 'Cox_Survival']
        survival_c_indices = []
        survival_names = []
        
        for model_name in survival_models:
            if model_name in self.results and 'Test' in self.results[model_name]:
                c_index = self.results[model_name]['Test'].get('c_index', np.nan)
                if not np.isnan(c_index):
                    survival_c_indices.append(c_index)
                    survival_names.append(model_name)
        
        if survival_c_indices:
            bars = axes[2].bar(survival_names, survival_c_indices, 
                             color=['skyblue', 'lightcoral'][:len(survival_names)])
            axes[2].set_title('생존 모델 성능 (C-index)')
            axes[2].set_ylabel('C-index')
            axes[2].set_ylim(0.5, 1.0)
            axes[2].grid(True, alpha=0.3, axis='y')
            
            for bar, value in zip(bars, survival_c_indices):
                axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 4. 치료효과 분류 모델 성능 (Accuracy)
        classification_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']
        classification_accuracies = []
        classification_names = []
        
        for model_name in classification_models:
            if model_name in self.results and 'Test' in self.results[model_name]:
                accuracy = self.results[model_name]['Test'].get('accuracy', np.nan)
                if not np.isnan(accuracy):
                    classification_accuracies.append(accuracy)
                    classification_names.append(model_name)
        
        if classification_accuracies:
            bars = axes[3].bar(classification_names, classification_accuracies,
                              color=['lightgreen', 'orange', 'gold'][:len(classification_names)])
            axes[3].set_title('치료효과 분류 모델 성능 (Accuracy)')
            axes[3].set_ylabel('Accuracy')
            axes[3].set_ylim(0, 1.0)
            axes[3].grid(True, alpha=0.3, axis='y')
            
            for bar, value in zip(bars, classification_accuracies):
                axes[3].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 5. 치료효과 분류 모델 성능 (AUC)
        classification_aucs = []
        for model_name in classification_names:
            auc_score = self.results[model_name]['Test'].get('auc', np.nan)
            classification_aucs.append(auc_score if not np.isnan(auc_score) else 0)
        
        if classification_aucs:
            bars = axes[4].bar(classification_names, classification_aucs, 
                              color=['lightgreen', 'orange', 'gold'][:len(classification_names)])
            axes[4].set_title('치료효과 분류 모델 성능 (AUC)')
            axes[4].set_ylabel('AUC')
            axes[4].set_ylim(0, 1.0)
            axes[4].grid(True, alpha=0.3, axis='y')
            
            for bar, value in zip(bars, classification_aucs):
                if value > 0:
                    axes[4].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 6-7. 특성 중요도 (XGBoost, LightGBM)
        for idx, model_name in enumerate(['XGB_Treatment', 'LGB_Treatment']):
            ax_idx = 5 + idx
            if model_name in self.models:
                try:
                    model_wrapper = self.models[model_name]
                    importance = model_wrapper['training_info']['feature_importance']
                    
                    feature_importance_df = pd.DataFrame({
                        'feature': self.feature_names,
                        'importance': importance
                    }).sort_values('importance', ascending=True).tail(10)
                    
                    color = 'lightcoral' if model_name == 'XGB_Treatment' else 'gold'
                    bars = axes[ax_idx].barh(range(len(feature_importance_df)), 
                                           feature_importance_df['importance'],
                                           color=color)
                    
                    axes[ax_idx].set_yticks(range(len(feature_importance_df)))
                    axes[ax_idx].set_yticklabels(feature_importance_df['feature'], fontsize=10)
                    axes[ax_idx].set_title(f'특성 중요도 ({model_name})', fontsize=12, fontweight='bold')
                    axes[ax_idx].set_xlabel('중요도')
                    axes[ax_idx].grid(True, alpha=0.3, axis='x')
                except Exception as e:
                    print(f"❌ {model_name} 특성 중요도 시각화 실패: {e}")
        
        # 8. 치료효과별 생존율 요약
        survival_summary = []
        time_points = [365, 1095, 1825]  # 1년, 3년, 5년
        
        for eff_level in [0, 1]:
            mask = (y_test_eff == eff_level)
            if mask.sum() > 5:
                kmf = KaplanMeierFitter()
                kmf.fit(y_test_dur[mask], y_test_event[mask])
                
                survival_rates = []
                for time_point in time_points:
                    try:
                        survival_rate = kmf.survival_function_at_times(time_point).values[0]
                        survival_rates.append(survival_rate * 100)
                    except:
                        survival_rates.append(np.nan)
                
                label = '치료효과 양호' if eff_level == 1 else '치료효과 불량'
                survival_summary.append({
                    'treatment_effectiveness': label,
                    'count': mask.sum(),
                    '1년': survival_rates[0],
                    '3년': survival_rates[1],
                    '5년': survival_rates[2]
                })
        
        # 생존율 테이블 시각화
        if survival_summary:
            survival_df = pd.DataFrame(survival_summary)
            table_data = []
            for _, row in survival_df.iterrows():
                table_data.append([
                    row['treatment_effectiveness'],
                    f"{row['count']}명",
                    f"{row['1년']:.1f}%" if not pd.isna(row['1년']) else "N/A",
                    f"{row['3년']:.1f}%" if not pd.isna(row['3년']) else "N/A",
                    f"{row['5년']:.1f}%" if not pd.isna(row['5년']) else "N/A"
                ])
            
            table = axes[7].table(cellText=table_data,
                                colLabels=['치료효과', '환자수', '1년 생존율', '3년 생존율', '5년 생존율'],
                                cellLoc='center',
                                loc='center',
                                colWidths=[0.2, 0.15, 0.2, 0.2, 0.2])
            table.auto_set_font_size(False)
            table.set_fontsize(10)
            table.scale(1, 2)
            axes[7].axis('off')
            axes[7].set_title('치료효과별 생존율 요약', fontweight='bold')
        
        # 9. CDSS 테스트 결과
        print("🔍 CDSS 테스트 결과 시각화 중...")
        cdss_results = self.test_cdss_compatibility()
        
        if cdss_results:
            cdss_table_data = []
            for model_name, result in cdss_results.items():
                if 'error' not in result:
                    if result.get('type') == 'classification':
                        cdss_table_data.append([
                            model_name,
                            result.get('class_name', 'N/A'),
                            f"{max(result.get('probabilities', [0])):.3f}" if 'probabilities' in result else 'N/A'
                        ])
                    else:
                        cdss_table_data.append([
                            model_name,
                            '생존 위험도 점수',
                            f"{result.get('risk_score', 0):.3f}"
                        ])
                else:
                    cdss_table_data.append([model_name, "오류", "N/A"])
            
            if cdss_table_data:
                cdss_table = axes[8].table(cellText=cdss_table_data,
                                         colLabels=['모델', '예측 결과', '확률/점수'],
                                         cellLoc='center',
                                         loc='center',
                                         colWidths=[0.4, 0.3, 0.3])
                cdss_table.auto_set_font_size(False)
                cdss_table.set_fontsize(10)
                cdss_table.scale(1, 2)
                axes[8].axis('off')
                axes[8].set_title('CDSS 호환성 테스트 결과 (Holdout 환자)', fontweight='bold')
        
        # 10-11. XAI 시각화 영역
        axes[9].text(0.5, 0.5, 'SHAP 시각화는 별도 파일로 저장됩니다\n(shap_summary_*.png, shap_waterfall_*.png)', 
                    ha='center', va='center', fontsize=14, transform=axes[9].transAxes)
        axes[9].set_title('SHAP 설명 가능 AI', fontweight='bold')
        axes[9].axis('off')
        
        axes[10].text(0.5, 0.5, 'LIME 시각화는 별도 파일로 저장됩니다\n(lime_explanation_*.png)', 
                     ha='center', va='center', fontsize=14, transform=axes[10].transAxes)
        axes[10].set_title('LIME 설명 가능 AI', fontweight='bold')
        axes[10].axis('off')
        
        plt.tight_layout()
        
        # XAI 시각화 생성
        shap_figures, lime_figures = self.generate_xai_visualizations(X_test, sample_index=0)
        
        main_results_path = "kidney_cancer_treatment_effect_analysis_results.png"
        plt.savefig(main_results_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✅ 메인 결과 저장: {main_results_path}")
        
        return main_results_path, shap_figures, lime_figures
    
    def generate_final_report(self):
        """최종 보고서 생성"""
        print("\n📋 최종 보고서 생성")
        
        report = f"""
{'='*80}
신장암 치료 효과 예측 모델 분석 보고서 (CDSS 호환 + XAI + 과적합 방지)
{'='*80}

📊 데이터 개요:
- 총 환자 수: {len(self.processed_df)}명
- 사망 환자: {self.processed_df['event'].sum()}명
- 사망률: {self.processed_df['event'].mean()*100:.1f}%
- 중간 추적 기간: {self.processed_df['duration'].median():.0f}일
- 치료효과 양호: {self.processed_df['treatment_effectiveness'].sum()}명

🎯 모델 성능 요약:
"""
        
        for model_name, results in self.results.items():
            report += f"\n{model_name}:\n"
            for dataset, metrics in results.items():
                if 'error' not in metrics:
                    if 'c_index' in metrics:
                        c_index = metrics.get('c_index', 'N/A')
                        report += f"  - {dataset}: C-index = {c_index:.3f}\n"
                    else:
                        accuracy = metrics.get('accuracy', 'N/A')
                        auc = metrics.get('auc', 'N/A')
                        if isinstance(accuracy, float):
                            report += f"  - {dataset}: Accuracy = {accuracy:.3f}, AUC = {auc:.3f}\n"
                        else:
                            report += f"  - {dataset}: Accuracy = {accuracy}, AUC = {auc}\n"
                else:
                    report += f"  - {dataset}: 오류 발생\n"
        
        report += f"""
🔬 CDSS 호환성:
- 모든 모델이 CDSS 호환 형태로 래핑됨
- Holdout 환자 테스트 완료
- 실시간 치료 효과 예측 가능

🧠 설명 가능 AI (XAI):
- SHAP 설명기 구현 완료
- LIME 설명기 구현 완료
- 특성 중요도 분석 완료

⚖️ 과적합 방지:
- 균형잡힌 하이퍼파라미터 설정
- 적절한 정규화 적용
- Early Stopping 구현

💾 저장된 파일:
- 모델 파일: cdss_kidney_cancer_treatment_*_model.pkl
- 전체 파이프라인: cdss_kidney_cancer_treatment_complete_pipeline.pkl
- 시각화 결과: kidney_cancer_treatment_effect_analysis_results.png
- XAI 시각화: shap_*.png, lime_*.png

⏰ 분석 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
{'='*80}
"""
        
        # 보고서 파일로 저장
        with open("kidney_cancer_treatment_effect_analysis_report.txt", "w", encoding='utf-8') as f:
            f.write(report)
        
        print(report)
        print("✅ 보고서 저장: kidney_cancer_treatment_effect_analysis_report.txt")
        
        return report
    
    def run_complete_analysis(self):
        """전체 분석 실행"""
        print("🚀 신장암 치료 효과 예측 모델 전체 분석 시작")
        print("="*80)
        
        try:
            # 1. 데이터 로드 및 탐색
            if not self.load_and_explore_data():
                return False
            
            # 2. 데이터 전처리
            if not self.preprocess_data():
                return False
            
            # 3. 특성 준비
            X, y_structured, y_duration, y_event, y_effectiveness = self.prepare_features()
            
            # 4. 데이터 분할
            (X_train, X_val, X_test, 
             y_train_struct, y_val_struct, y_test_struct,
             y_train_dur, y_val_dur, y_test_dur,
             y_train_event, y_val_event, y_test_event,
             y_train_eff, y_val_eff, y_test_eff) = self.split_data(
                X, y_structured, y_duration, y_event, y_effectiveness)
            
            # 5. 모델 훈련
            if not self.train_models(X_train, X_val, X_test,
                                   y_train_struct, y_val_struct, y_test_struct,
                                   y_train_dur, y_val_dur, y_test_dur,
                                   y_train_event, y_val_event, y_test_event,
                                   y_train_eff, y_val_eff, y_test_eff):
                return False
            
            # 6. 모델 평가
            if not self.evaluate_models(X_train, X_val, X_test,
                                       y_train_struct, y_val_struct, y_test_struct,
                                       y_train_dur, y_val_dur, y_test_dur,
                                       y_train_event, y_val_event, y_test_event,
                                       y_train_eff, y_val_eff, y_test_eff):
                return False
            
            # 7. XAI 설명 생성
            if not self.explain_models(X_train, X_test):
                print("⚠️ XAI 설명 생성 실패, 계속 진행")
            
            # 8. 결과 시각화
            main_plot, shap_plots, lime_plots = self.plot_results(
                X_test, y_test_dur, y_test_event, y_test_eff)
            
            # 9. CDSS 호환 모델 저장
            if not self.save_models_for_cdss():
                print("⚠️ 모델 저장 실패, 계속 진행")
            
            # 10. 최종 보고서 생성
            final_report = self.generate_final_report()
            
            print("\n🎉 신장암 치료 효과 예측 모델 분석 완료!")
            print("✅ 모든 결과가 저장되었습니다.")
            
            return True
            
        except Exception as e:
            print(f"\n❌ 분석 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            return False

# 사용 예시
if __name__ == "__main__":
    # 데이터 파일 경로 설정
    data_path = r"G:\.shortcut-targets-by-id\1aXfYtUWSYS8foz14MAJMhDH7IHhG1wNZ\2조\데이터\clinical model\kidney\TCGA-KIRP_clinical_data.csv"  # 실제 데이터 파일 경로로 변경
    
    # 분석 객체 생성 및 실행
    analyzer = KidneyCancerTreatmentEffectPredictorCDSS(data_path)
    success = analyzer.run_complete_analysis()
    
    if success:
        print("\n🎯 분석 결과 요약:")
        print("- 5개의 CDSS 호환 치료 효과 예측 모델 생성 완료")
        print("  * Random Survival Forest (생존 예측)")
        print("  * Cox 비례위험모델 (생존 예측)")
        print("  * Random Forest (치료 효과 분류)")
        print("  * XGBoost (치료 효과 분류)")
        print("  * LightGBM (치료 효과 분류)")
        print("- XAI 설명 가능 AI 구현 완료")
        print("- 과적합 방지 최적화 완료")
        print("- 실시간 치료 효과 예측 시스템 준비 완료")
        print("- 모든 결과 파일 저장 완료")
    else:
        print("\n❌ 분석 실패")


c:\Users\02\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 13.0.0. Please consider upgrading.
  warnings.warn(


✅ 한글 폰트 설정 완료
🚀 신장암 치료 효과 예측 모델 초기화 (CDSS 호환 + XAI + 과적합 방지)
📁 데이터 경로: G:\.shortcut-targets-by-id\1aXfYtUWSYS8foz14MAJMhDH7IHhG1wNZ\2조\데이터\clinical model\kidney\TCGA-KIRP_clinical_data.csv
⏰ 시작 시간: 2025-06-20 12:45:05
🚀 신장암 치료 효과 예측 모델 전체 분석 시작

📊 1. 데이터 로드 및 탐색
✅ 데이터 로드 성공: 291행 × 97열
📈 데이터 기본 정보:
   - 총 환자 수: 291
   - 총 컬럼 수: 97
   - 생존 환자: 247명
   - 사망 환자: 44명
   - 사망률: 15.1%

🔧 2. 데이터 전처리
✅ 사용 가능한 컬럼: 28개
🔄 생존 변수 생성 중...
✅ 유효한 생존 데이터: 288명
   - 사망 이벤트: 44건
   - 중간 생존 시간: 768일
   - 치료 효과 양호: 145명

📋 결측값 분석:
   - days_to_death: 244개 (84.7%)
   - treatments_pharmaceutical_treatment_type: 78개 (27.1%)
   - treatments_pharmaceutical_treatment_intent_type: 79개 (27.4%)
   - treatments_pharmaceutical_treatment_or_therapy: 78개 (27.1%)
   - treatments_radiation_treatment_type: 63개 (21.9%)
   - treatments_radiation_treatment_or_therapy: 63개 (21.9%)
   - ajcc_pathologic_stage: 30개 (10.4%)
   - ajcc_pathologic_n: 1개 (0.3%)
   - ajcc_pathologic_m: 15개 (5.2%)
   - age_at_diagnosis: 26개 (9.0%)
   -